In [47]:
#imports
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import glob

random.seed(108)

In [45]:
#Classes for data 
class NumberOfBus:
    def __init__(self, filename, size, bndbox):
        self.filename = filename
        self.size = Size(size)
        self.bndbox = BndBox(bndbox)

    @staticmethod
    def extract_fields_from_xml(xml_file):
        root = ET.parse(xml_file).getroot()
        filename = ''
        size = ''
        bbox = ''
        # Parse the XML Tree
        for elem in root:
            # Get the file name 
            if elem.tag == "filename":
                filename = elem.text
                
            # Get the image size
            elif elem.tag == "size":
                size = elem

            # Get details of the bounding box 
            elif elem.tag == "object":
                for subelem in elem:
                    if subelem.tag == "bndbox":
                        bbox = subelem
        return NumberOfBus(filename, size, bbox)

class Size:
    def __init__(self, xmlNode):
        width = 0
        height = 0
        depth = 0
        for node in xmlNode:
            if node.tag == "width":
                width = int(node.text)
            elif node.tag == "height":
                height = int(node.text)
            elif node.tag == "depth":
                depth = int(node.text)
        self.width = width
        self.height = height
        self.depth  = depth

class BndBox:
    def __init__(self, xmlNode) -> None:
        xmax = 0
        xmin = 0
        ymin = 0
        ymax = 0
        for node in xmlNode:
            if node.tag == "xmax":
                xmax = int(node.text)
            elif node.tag == "xmin":
                xmin = int(node.text)
            elif node.tag == "ymin":
                ymin = int(node.text)
            elif node.tag == "ymax":
                ymax = int(node.text)
        self.xmax = xmax
        self.xmin = xmin
        self.ymin = ymin
        self.ymax = ymax 

In [43]:
xml_file = ('DataXml\\test\\image-1-53_jpg.rf.acf7c6c8961a2e16284a42187fa13aff.xml')
obj = NumberOfBus.extract_fields_from_xml(xml_file)

In [50]:
os.listdir("DataXml\\test")
files = glob.glob(os.path.join('DataXml\\test', '*.xml'))
list_bus_data = []
for filePath in files: 
    list_bus_data.append(NumberOfBus.extract_fields_from_xml(filePath))
print(list_bus_data)

[<__main__.NumberOfBus object at 0x000001771DAFE050>, <__main__.NumberOfBus object at 0x000001771F2880D0>, <__main__.NumberOfBus object at 0x000001771E3FF5D0>, <__main__.NumberOfBus object at 0x000001771E7795D0>, <__main__.NumberOfBus object at 0x000001771E778110>, <__main__.NumberOfBus object at 0x000001771E7A0C50>, <__main__.NumberOfBus object at 0x000001771E7793D0>, <__main__.NumberOfBus object at 0x000001771E77AA10>, <__main__.NumberOfBus object at 0x000001771E3FFE90>, <__main__.NumberOfBus object at 0x000001771E7A1FD0>, <__main__.NumberOfBus object at 0x000001771DAE8610>, <__main__.NumberOfBus object at 0x000001771E77A6D0>, <__main__.NumberOfBus object at 0x000001771E779110>, <__main__.NumberOfBus object at 0x000001771E77A310>, <__main__.NumberOfBus object at 0x000001771E779350>, <__main__.NumberOfBus object at 0x000001771E77A650>, <__main__.NumberOfBus object at 0x000001771F289D50>, <__main__.NumberOfBus object at 0x000001771E743590>, <__main__.NumberOfBus object at 0x000001771E7

In [ ]:
# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(busData: list):
   
    # For each bounding box
    for busItem in busData:
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (busItem.xmin + busItem.xmax) / 2 
        b_center_y = (busItem.ymin + busItem.ymax) / 2
        b_width    = (busItem.xmax - busItem.xmin)
        b_height   = (busItem.ymax - busItem.ymin)
        
        # Normalise the co-ordinates by the dimensions of the image
        b_center_x /= busItem.size.width
        b_center_y /= busItem.size.height
        b_width    /= busItem.size.width
        b_height   /= busItem.size.height 
        
        #Write the bbox details to the file 
        print_buffer.append("{:.3f} {:.3f} {:.3f} {:.3f}".format(b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk 
    print("\n".join(print_buffer), file= open(save_file_name, "w"))